<a href="https://colab.research.google.com/github/BlessingBassey/masakhane/blob/master/efi_en_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "efi"
target_language = "en" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/efi-en-baseline


In [0]:
#TODO: Skip for retrain
# Install opus-tools
! pip install opustools-pkg 

     |████████████████████████████████| 81kB 2.6MB/s 


In [0]:
#TODO: Skip for retrain
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/efi-en.xml.gz not found. The following files are available for downloading:

   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/efi-en.xml.gz
  36 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/efi.zip
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip

 303 MB Total size
./JW300_latest_xml_efi-en.xml.gz ... 100% of 3 MB
./JW300_latest_xml_efi.zip ... 100% of 36 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB


In [0]:
# extract the corpus file
! gunzip JW300_latest_xml_$tgt-$src.xml.gz

gzip: JW300_latest_xml_en-efi.xml.gz: No such file or directory


In [0]:
#TODO: Skip for retrain
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-04-07 10:41:12--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.1s    

2020-04-07 10:41:13 (2.35 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-04-07 10:41:15--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-efi.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [0]:
#TODO: Skip for retrain
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
#TODO: Skip for retrain
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 6113/377824 lines since contained in test set.


,source_sentence,target_sentence
0,© 2013 Watch Tower Bible and Tract Society of ...,© 2013 Watch Tower Bible and Tract Society of ...
1,All rights reserved .,All rights reserved .
2,3 Watching the World,3 Se Itịbede ke Ererimbot


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
#TODO: Skip for retrain
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
#TODO: Skip for retrain
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 911kB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144801 sha256=9a3225ef63aa0c469b1c17d2027ee01d92c08cfa2ceb1c887a5348413e5ae974
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.13 0.00 percent complete


00:00:23.47 0.30 percent complete
00:00:47.23 0.59 percent complete
00:01:13.23 0.89 percent complete
00:01:37.25 1.19 percent complete
00:02:00.91 1.48 percent complete
00:02:25.30 1.78 percent complete
00:02:48.69 2.08 percent complete
00:03:12.44 2.37 percent complete
00:03:36.68 2.67 percent complete
00:04:00.63 2.97 percent complete
00:04:26.31 3.26 percent complete
00:04:50.18 3.56 percent complete
00:05:14.62 3.86 percent complete
00:05:38.29 4.15 percent complete
00:06:01.78 4.45 percent complete
00:06:25.15 4.75 percent complete
00:06:48.09 5.04 percent complete
00:07:11.21 5.34 percent complete
00:07:35.76 5.64 percent complete
00:07:58.99 5.93 percent complete
00:08:22.42 6.23 percent complete
00:08:46.08 6.53 percent complete


00:09:09.18 6.82 percent complete
00:09:32.55 7.12 percent complete
00:09:55.41 7.42 percent complete


00:10:19.19 7.71 percent complete
00:10:44.31 8.01 percent complete
00:11:07.99 8.31 percent complete
00:11:31.20 8.60 percent complete
00:11:55.25 8.90 percent complete
00:12:18.14 9.20 percent complete
00:12:41.67 9.49 percent complete
00:13:05.32 9.79 percent complete
00:13:28.60 10.09 percent complete
00:13:53.16 10.38 percent complete


00:14:17.50 10.68 percent complete
00:14:40.37 10.98 percent complete
00:15:03.19 11.27 percent complete
00:15:25.76 11.57 percent complete
00:15:48.21 11.87 percent complete
00:16:09.87 12.16 percent complete
00:16:32.29 12.46 percent complete
00:16:54.84 12.76 percent complete
00:17:17.29 13.05 percent complete
00:17:39.13 13.35 percent complete
00:18:01.50 13.65 percent complete
00:18:23.71 13.94 percent complete
00:18:45.92 14.24 percent complete
00:19:08.43 14.54 percent complete
00:19:31.39 14.83 percent complete
00:19:54.26 15.13 percent complete
00:20:17.94 15.43 percent complete
00:20:40.82 15.72 percent complete
00:21:03.51 16.02 percent complete
00:21:26.55 16.32 percent complete
00:21:49.64 16.61 percent complete
00:22:13.80 16.91 percent complete
00:22:37.37 17.21 percent complete
00:23:00.26 17.50 percent complete
00:23:25.15 17.80 percent complete
00:23:48.26 18.10 percent complete
00:24:11.21 18.39 percent complete
00:24:34.04 18.69 percent complete
00:24:56.74 18.99 pe

...

01:22:51.12 64.08 percent complete
01:23:14.11 64.37 percent complete
01:23:37.09 64.67 percent complete
01:24:00.07 64.97 percent complete
01:24:23.24 65.26 percent complete
01:24:46.84 65.56 percent complete
01:25:11.42 65.86 percent complete
01:25:34.14 66.15 percent complete
01:25:57.85 66.45 percent complete
01:26:21.15 66.75 percent complete
01:26:44.18 67.04 percent complete
01:27:06.11 67.34 percent complete
01:27:28.17 67.64 percent complete
01:27:50.68 67.93 percent complete
01:28:16.04 68.23 percent complete
01:28:38.68 68.53 percent complete
01:29:01.66 68.82 percent complete


01:29:23.92 69.12 percent complete
01:29:47.07 69.42 percent complete
01:30:10.07 69.71 percent complete
01:30:34.07 70.01 percent complete
01:30:57.53 70.31 percent complete
01:31:24.58 70.60 percent complete
01:31:47.52 70.90 percent complete
01:32:10.81 71.20 percent complete
01:32:33.92 71.49 percent complete
01:32:57.01 71.79 percent complete
01:33:18.97 72.09 percent complete
01:33:41.68 72.38 percent complete
01:34:04.01 72.68 percent complete
01:34:29.55 72.98 percent complete
01:34:53.10 73.27 percent complete
01:35:16.17 73.57 percent complete


01:35:39.54 73.87 percent complete
01:36:02.24 74.16 percent complete
01:36:25.48 74.46 percent complete
01:36:48.49 74.76 percent complete
01:37:10.46 75.05 percent complete
01:37:36.30 75.35 percent complete
01:37:59.14 75.65 percent complete
01:38:22.44 75.94 percent complete
01:38:44.61 76.24 percent complete
01:39:06.57 76.54 percent complete
01:39:29.21 76.83 percent complete
01:39:52.37 77.13 percent complete
01:40:15.33 77.43 percent complete


01:40:41.48 77.72 percent complete
01:41:04.55 78.02 percent complete
01:41:27.18 78.32 percent complete
01:41:51.03 78.61 percent complete
01:42:15.09 78.91 percent complete
01:42:39.00 79.21 percent complete
01:43:02.63 79.50 percent complete
01:43:25.90 79.80 percent complete
01:43:51.44 80.10 percent complete
01:44:14.54 80.39 percent complete
01:44:37.70 80.69 percent complete
01:45:01.10 80.99 percent complete
01:45:24.57 81.28 percent complete
01:45:48.03 81.58 percent complete
01:46:11.43 81.88 percent complete
01:46:34.64 82.17 percent complete
01:47:00.82 82.47 percent complete
01:47:23.07 82.77 percent complete
01:47:46.29 83.06 percent complete
01:48:08.38 83.36 percent complete
01:48:31.21 83.66 percent complete
01:48:53.93 83.95 percent complete
01:49:17.20 84.25 percent complete
01:49:39.98 84.55 percent complete
01:50:05.94 84.84 percent complete
01:50:28.82 85.14 percent complete
01:50:51.72 85.44 percent complete
01:51:15.22 85.73 percent complete
01:51:37.48 86.03 pe

01:59:27.71 91.96 percent complete
01:59:53.34 92.26 percent complete
02:00:15.96 92.56 percent complete
02:00:38.31 92.85 percent complete
02:01:01.63 93.15 percent complete
02:01:25.17 93.45 percent complete
02:01:47.62 93.74 percent complete
02:02:10.33 94.04 percent complete
02:02:33.31 94.34 percent complete
02:02:57.88 94.63 percent complete
02:03:19.49 94.93 percent complete
02:03:42.46 95.23 percent complete
02:04:04.82 95.52 percent complete
02:04:27.72 95.82 percent complete
02:04:50.36 96.12 percent complete
02:05:13.34 96.41 percent complete
02:05:35.17 96.71 percent complete
02:06:00.93 97.01 percent complete
02:06:23.25 97.30 percent complete
02:06:45.72 97.60 percent complete
02:07:07.80 97.89 percent complete
02:07:30.31 98.19 percent complete
02:07:52.66 98.49 percent complete
02:08:14.26 98.78 percent complete
02:08:36.20 99.08 percent complete
02:09:01.21 99.38 percent complete
02:09:23.58 99.67 percent complete
02:09:46.32 99.97 percent complete


In [0]:
#TODO: Skip for retrain
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.efi <==
Isaiah 9 : 7 ọdọho ke Eyen Abasi edidi Edidem ye nte ke enye ayanam ediwak nti n̄kpọ ọnọ ubonowo . “ Ifịk Jehovah mme udịm edinam emi . ”
The New Encyclopædia Britannica ọdọhọ ke Mme Ntiense Jehovah “ ẹdu uwem nte Bible etemede . ”
Mmọ ẹkesụk ẹdu ke ini emi wheat ye mbiet ẹkọride ọtọkiet , ndien owo ikokụreke kan̄a ndutịm oro ẹkenamde man ẹnyene mbon emi ẹdisinọde mme owo udia eke spirit .
SIO INI NỊM NDINAM ITIE UFAN ỌKỌRI .
Ndien ami nyeben̄e ekụri nsiak ifia nnịm nnọ enye edida etem udia .
Ini kiet , mma ntọhọ nnyụn̄ n̄n̄wana ye owo unek emi eketiede ubi ubi , nnyụn̄ mmia unamikọt nsio ntop nduọk ko !
Edi Andibot ọmọn̄wọn̄ọ ete ke imọ iyọsọp ida utịt isọk ererimbot n̄kaowo oro odude ke emi ke idak ukara Satan kpa Devil .
Ami ye Roy ima idomo ndidu uwem ekekem ye enyịn̄ oro ebe ke ndibuana ke kpukpru usụn̄ ukwọrọikọ ye ubịnikọt oro esop ekesịnde udọn̄ ọnọ .
T .
Sylvia emi edide nurse ọdọhọ ete : “ Ediwak mbon oro ikakade n̄wed ntre ẹma ẹsika ufọkabasi .

==> train.e



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [4]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2380 (delta 0), reused 0 (delta 0), pack-reused 2377
Receiving objects: 100% (2380/2380), 2.60 MiB | 1.70 MiB/s, done.
Resolving deltas: 100% (1670/1670), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 276kB 9.5MB/s 
     |████████████████████████████████| 307kB 39.8MB/s 
     |████████████████████████████████| 17.1MB 204kB/s 
     |████████████████████████████████| 51kB 9.7MB/s 
     |████████████████████████████████| 215kB 61.2MB/s 
     |████████████████████████████████| 747kB 53.4MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73768 sha256=800dfbc5f682857686d8708ff7bcf7db31d2634b9fd6befdf04b750ce0dada12
  Stored in directory: /tmp/pip-ephem-wheel-cac

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
#TODO: Skip for retrain
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path/vocab.txt"  # Herman

bpe.codes.4000	dev.efi       test.bpe.en  test.en-any.en  train.efi
dev.bpe.efi	dev.en	      test.efi	   train.bpe.efi   train.en
dev.bpe.en	test.bpe.efi  test.en	   train.bpe.en
1000.hyps  4000.hyps	   dev.efi	  test.bpe.en	    train.bpe.en
2000.ckpt  best.ckpt	   dev.en	  test.efi	    train.efi
2000.hyps  bpe.codes.4000  models	  test.en	    train.en
3000.ckpt  config.yaml	   src_vocab.txt  test.en-any.en    train.log
3000.hyps  dev.bpe.efi	   tensorboard	  test.en-any.en.1  trg_vocab.txt
4000.ckpt  dev.bpe.en	   test.bpe.efi   train.bpe.efi     validations.txt
BPE Xhosa Sentences
18 , 19 . ( a ) Didie ke nditọete ke esop mbufo ẹkeme ndin̄wam fi ada san̄asan̄a ?
“ Ndi@@ tie n̄kere se Mme N̄ke 27 : 11 , Matthew 26 : 5@@ 2 , ye John 13 : 35 ẹdọhọde ama an̄wam mi nt@@ etịm mb@@ iere ke ndid@@ ụk@@ ke ekọn̄ .
Mme itie N̄wed Abasi emi ama anam esịt ana mi sụn̄ ke ini afanikọn̄ emi . ” — A@@ nd@@ ri@@ y emi otode Uk@@ ra@@ ine .
“ Isaiah 2 : 4 ama an̄wam mi n̄ka iso nda san̄asan̄a ke ini i

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [6]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

Last checkpoint : best.ckpt


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # TODO: uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 50                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [15]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

Streaming output truncated to the last 5000 lines.
2020-04-10 13:49:25,870 	Hypothesis: The prayer may be the case of the clergy to them .
2020-04-10 13:49:25,870 Example #2
2020-04-10 13:49:25,871 	Source:     Ẹtịn̄ ukem ikọ oro ke 2 Chronicles 5 : 9 .
2020-04-10 13:49:25,871 	Reference:  The same point is made at 2 Chronicles 5 : 9 .
2020-04-10 13:49:25,871 	Hypothesis: The same words at 2 Chronicles 5 : 9 .
2020-04-10 13:49:25,871 Example #3
2020-04-10 13:49:25,872 	Source:     Ẹkọbi Paul ẹtem ke ufọk esie ke Rome ke isua iba ( ke n̄kpọ nte isua 59 esịm 61 E.N . ) , ndien enye oyom usụn̄ do ọkwọrọ Obio Ubọn̄ onyụn̄ ekpep mbon en̄wen “ mme n̄kpọ emi ẹban̄ade Ọbọn̄ Jesus Christ . ” — Utom 28 : 30 , 31 .
2020-04-10 13:49:25,872 	Reference:  59 - 61 C.E . ) , and from there he finds ways to preach about the Kingdom and teach “ the things concerning the Lord Jesus Christ . ” ​ — Acts 28 : 30 , 31 .
2020-04-10 13:49:25,872 	Hypothesis: Paul was reminded in his home in Rome in two years ( 

In [0]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [17]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 111572.57812	PPL: 74.63804	bleu: 1.35220	LR: 0.00030000	*
Steps: 2000	Loss: 97350.62500	PPL: 43.07422	bleu: 3.17081	LR: 0.00030000	*
Steps: 3000	Loss: 88266.82031	PPL: 30.31993	bleu: 5.37002	LR: 0.00030000	*
Steps: 4000	Loss: 81728.03906	PPL: 23.54844	bleu: 7.36726	LR: 0.00030000	*
Steps: 5000	Loss: 77013.80469	PPL: 19.62568	bleu: 8.87327	LR: 0.00030000	*
Steps: 6000	Loss: 73350.49219	PPL: 17.03449	bleu: 10.38091	LR: 0.00030000	*
Steps: 7000	Loss: 70315.99219	PPL: 15.14918	bleu: 12.35965	LR: 0.00030000	*
Steps: 8000	Loss: 67509.00781	PPL: 13.59153	bleu: 13.74374	LR: 0.00030000	*
Steps: 9000	Loss: 65475.03125	PPL: 12.56390	bleu: 14.65290	LR: 0.00030000	*
Steps: 10000	Loss: 63727.16406	PPL: 11.74311	bleu: 15.95746	LR: 0.00030000	*
Steps: 11000	Loss: 62284.03906	PPL: 11.10599	bleu: 16.49099	LR: 0.00030000	*
Steps: 12000	Loss: 61020.59766	PPL: 10.57665	bleu: 16.96966	LR: 0.00030000	*
Steps: 13000	Loss: 59996.48047	PPL: 10.16614	bleu: 17.70745	LR: 0.00030000	*
Steps: 14000

In [18]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"


2020-04-10 22:01:59,541 Hello! This is Joey-NMT.
2020-04-10 22:02:30,804  dev bleu:  28.54 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-04-10 22:03:11,353 test bleu:  33.68 [Beam search decoding with beam size = 5 and alpha = 1.0]
